In [116]:
import zipfile
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy import stats
from datetime import datetime as dt, timedelta
from IPython.display import display, HTML
%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import functions as f

In [2]:
filename = './data/Master_Data_Union_2014_2018_B_Recency_Pre2014.txt'

In [5]:
#archive = zipfile.ZipFile('./data/Master_Data_Union_2014_2018_B_Recency_Pre2014.zip', 'r')

In [3]:
%%time
chunksize = 10**6
chunks = pd.read_csv(filename, chunksize=chunksize, sep='|',low_memory=False)
#for chunk in pd.read_csv(filename,chunksize=chunksize,sep='|',low_memory=False):
    #DF = pd.concat([DF,chunk])

CPU times: user 4.77 ms, sys: 5.09 ms, total: 9.85 ms
Wall time: 26.6 ms


In [283]:
%%time
df = chunks.get_chunk(1000000)

CPU times: user 15.8 s, sys: 6.77 s, total: 22.6 s
Wall time: 26.3 s


In [ ]:
#only selecting orders that happened between may 2017-july 2017 for sample size

In [284]:
%%time
DF_SAMPLE = df.loc[df.IS_EMAILABLE_IND=='Y']
df1 = DF_SAMPLE.copy()
df1['ORDER_DATE'] = df1.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d'))
df1 = df1.loc[(df1.ORDER_DATE >= dt(2017,5,1).date()) & 
              (df1.ORDER_DATE < dt(2017,8,1).date())]
#df1 = df1.loc[(df1.ORDER_DATE.dt.month >= 5) & (df1.ORDER_DATE.dt.month < 8)]
df1 = df1[~df1.BIRTH_DATE.isnull()]

CPU times: user 14.1 s, sys: 3.32 s, total: 17.4 s
Wall time: 21.2 s


In [285]:
print 'unique customer ids:', len(df1.ILINK.unique())
print 'Number emailable:', df1.shape[0]

unique customer ids: 5597
Number emailable: 40180


In [286]:
df1['BIRTH_DATE'] = df1.BIRTH_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_CATALOG_DATE'] = df1.FIRST_CATALOG_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['ORDER_DATE'] = df1.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
df1['FIRST_ORDER_DATE'] = df1.FIRST_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_RETAIL_ORDER_DATE'] = df1.FIRST_RETAIL_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_WEB_ORDER_DATE'] = df1.FIRST_WEB_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_CLOSE_DATE'] = df1.JJCH_CLOSE_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_OPEN_DATE'] = df1.JJCH_OPEN_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['PRIOR_ORDER_DATE'] = df1.PRIOR_ORDER_DATE.apply(lambda x: dt.strptime(x,'%m/%d/%Y').date())

In [287]:
ilinks = df1.ILINK.unique().tolist()

# Definitions

In [288]:
def normalize(l):
    s = sum(l)
    if s == 0:
        return [0]*8
    else:
        return [i/s for i in l]

##standardize variables
def standardization_f(x):
    x_bar = np.mean(x)
    s = np.std(x)
    if s == 0:
        return 0
    else:
        x_z = (x-x_bar)/s
    return(x_z)

In [289]:
#make empty dataframe to add onto with customer features
columns = ['Age',
           'WovenShirts_N','KnitTops_N','Pants_N','Dresses_N',
           'WovenShirts_AvgN_Order','KnitTops_AvgN_Order','Pants_AvgN_Order','Dresses_AvgN_Order',
           'WovenShirts_Total','KnitTops_Total','Pants_Total','Dresses_Total',
           'WovenShirts_Amt_Order','KnitTops_Amt_Order','Pants_Amt_Order','Dresses_Amt_Order',
           'WovenShirts_AvgAmt','KnitTops_AvgAmt','Pants_AvgAmt','Dresses_AvgAmt']


# Exploration


In [410]:
reload(f)

<module 'functions' from 'functions.py'>

In [420]:
cols = ['Ilink','N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
cols2 = ['Ilink','N_ws','S_ws','s_ws','si_ws','R_ws',
        'N_kt','S_kt','s_kt','si_kt','R_kt',
        'N_d','S_d','s_d','si_d','R_d',
        'N_p','S_p','s_p','si_p','R_p']
table = pd.DataFrame([],columns=cols)

In [421]:
%%time
#need to create rows that describe a customers freq and sales of items in specific departments
depts = ['Woven Shirts','Knit Tops','Dresses','Pants']
K = 4 #Number of Depts/topics
for ilink in ilinks[:20]:
    dfCust = df1.loc[(df1.ILINK==ilink)]
    N_u = dfCust.shape[0]#sum([f.getTotalFreq(dfCust,dept) for dept in depts])
    fts = []
    #fts2 = []
    for dept in depts:
        N = f.getTotalFreq(dfCust,dept)
        n = f.getAvgFreqOrder(dfCust,dept)
        O,Q,G,S,D = f.getTotalAmts(dfCust,dept)
        o,q,g,s,d = f.getAvgAmtsOrder(dfCust,dept)
        oi,qi,gi,si,di = f.getAvgAmtItem(dfCust,dept)
        alpha = [N,n,S,s,si,D,d,di]
        #alpha = [N,S,s,si]
        alpha = [0 if math.isnan(x) else x for x in alpha]
        alpha1 = np.array(normalize(alpha))
        sigma = (N + alpha1)/(N_u + K*alpha1)
        pref = np.dot(sigma,alpha1).round(2)
        #print ilink, alpha1
        #print sigma
        #print ''
        fts.append(alpha+[pref])
        #fts2.append(alpha)
    
    """u,s,vt = np.linalg.svd(fts2)
    for i,row in enumerate(fts2):
        fts2[i] = fts2[i] + [np.dot(u[i],vt[i]).round(3)]"""
    row = [i for sub in fts for i in sub]
    row.insert(0,ilink)
    temp = pd.DataFrame([tuple(row)],columns=cols)
    table = pd.concat([table,temp])

CPU times: user 1.78 s, sys: 33.6 ms, total: 1.81 s
Wall time: 1.82 s


In [422]:
table.sort_values('R_p',ascending=False)

,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
0,50433,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0,2,2.0,69.98,69.98,34.99,30.00,30.00,15.00,0.79
0,50192,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0,1,1.0,46.54,46.54,46.54,23.45,23.45,23.45,0.69
0,50285,0,0.0,0.00,0.00,0,0.00,0.00,0,0,1,1.0,30.80,30.80,30.8,13.20,13.20,13.2,0.25,0,0.0,0.00,0.00,0,0.00,0.00,0,0,3,3.0,123.90,123.90,41.30,53.10,53.10,17.70,0.66
0,50220,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0,2,0.0,118.98,23.80,59.49,21.00,4.20,10.5,0.14,9,2.0,412.50,82.50,45.83,158.50,31.70,17.61,0.57
0,50413,1,0.0,48.30,5.37,48.3,20.70,2.30,20.7,0.1,4,0.0,149.25,16.58,37.31,36.75,4.08,9.19,0.32,0,0.0,0.00,0.00,0,0.00,0.00,0,0,6,1.0,293.53,32.61,48.92,132.45,14.72,22.07,0.47
0,50354,0,0.0,0.00,0.00,0,0.00,0.00,0,0,2,1.0,95.25,31.75,47.63,12.75,4.25,6.38,0.44,0,0.0,0.00,0.00,0,0.00,0.00,0,0,2,1.0,104.75,34.92,52.38,33.25,11.08,16.63,0.45
0,50260,0,0.0,0.00,0.00,0,0.00,0.00,0,0,3,1.0,112.98,37.66,37.66,47.01,15.67,15.67,0.36,0,0.0,0.00,0.00,0,0.00,0.00,0,0,3,1.0,169.01,56.34,56.34,77.99,26.00,26.00,0.36
0,50276,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0,1,0.0,56.99,28.50,56.99,3.00,1.50,3.00,0.31
0,50280,0,0.0,0.00,0.00,0,0.00,0.00,0,0,1,1.0,20.99,20.99,20.99,9.00,9.00,9,0.25,1,1.0,34.99,34.99,34.99,15.00,15.00,15,0.25,1,1.0,41.99,41.99,41.99,18.00,18.00,18.00,0.25
0,50198,1,0.0,52.53,6.57,52.53,26.47,3.31,26.47,0.06,8,1.0,305.78,38.22,38.22,72.21,9.03,9.03,0.37,6,1.0,456.28,57.04,76.05,47.72,5.96,7.95,0.28,3,0.0,147.58,18.45,49.19,69.42,8.68,23.14,0.15


In [417]:
items = []
for order in df1[df1.ILINK == 50220].ORDER_KEY.unique():
    temp = df1[(df1.ILINK == 50220) & (df1.ORDER_KEY==order) ]
    x = temp.groupby('DEPARTMENT_NAME').size().to_dict()
    if 'Dresses' in x:
        items.append(x['Dresses'])
    else:
        items.append(0)
print np.mean(items)

0.4
